In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
%%capture
!pip install datasets
!pip install transformers==4.17 sentencepiece
!pip install rouge_score
!pip install transformers[torch]
!pip install accelerate -U
!pip install evaluate
!pip install wandb

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
from datasets import Dataset
import json
import os
import pandas as pd
from datasets import load_dataset

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("minnehwg/finetune-newwiki-summarization-ver1")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/741 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/862M [00:00<?, ?B/s]

### Read and pre-processing the data

In [7]:
file_path = '/kaggle/input/newwiki-pandas/data.pkl'
df = pd.read_pickle(file_path)

from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [8]:
dataset[2]

{'inputs': 'Một số loại hình dán có keo dính ở đằng sau, số khác chỉ có một miếng vải bên dưới. Xem xét kỹ hình dán và xác định xem bạn có cần thêm vật liệu gì không. Những hình dán thêu trang trí thường dày, cứng và có một lớp keo nhựa ở mặt dưới. Loại này có thể dùng để che giấu phần vải bị rách hoặc mất màu. Hình dán truyền nhiệt có hình in trên một mặt giấy, và mặt kia là loại giấy không bóng. Loại này không thể vá được chỗ rách, và phần vải bên dưới thường bị lộ rõ nếu không phải là vải trắng. Hình dán có mặt sau chỉ là chất liệu vải có thể gắn bằng vải keo ép. Các hình dán dùng để vá các lỗ thủng hoặc vết ố bẩn được thiết kế để hòa lẫn với vải thường có một lớp giấy đằng sau và cần được gỡ ra trước khi dán. Cân nhắc tự thiết kế hình dán nếu bạn không tìm được mẫu yêu thích. Các chất liệu như denim và cotton thường làm nền rất tốt cho các hình dán bằng bàn là. Nguyên tắc chung là chất liệu vải ít nhất phải dày bằng hình dán. Xem nhãn hướng dẫn cách chăm sóc quần áo để biết chất li

In [9]:
# import re

# def clean_text(example):
# #     example['inputs'] = re.sub(r'[^a-zA-Z0-9\s.,!?;:]', ' ', example['inputs'])
# #     example['labels'] = re.sub(r'[^a-zA-Z0-9\s.,!?;:]', ' ', example['labels'])
#     example['inputs'] = re.sub(r'\s+', ' ', example['inputs']).strip()
#     example['labels'] = re.sub(r'\s+', ' ', example['labels']).strip()
#     return example

import unicodedata

def clean_text(example):
    # Loại bỏ các ký tự không mong muốn từ chuỗi inputs và labels
    example['inputs'] = remove_unwanted_chars(example['inputs'])
    example['labels'] = remove_unwanted_chars(example['labels'])
    
    # Xóa khoảng trắng dư thừa trong inputs và labels, và loại bỏ khoảng trắng ở đầu và cuối chuỗi
    example['inputs'] = ' '.join(example['inputs'].split())
    example['labels'] = ' '.join(example['labels'].split())
    
    return example

def remove_unwanted_chars(text):
    cleaned_text = []
    for char in text:
        # Kiểm tra xem ký tự có phải là ký tự chữ cái, chữ số, hoặc dấu cụ nhất định không
        category = unicodedata.category(char)
#         or category.startswith('P')
        if char.isalnum() or char in [' ', '.','!', '?', ';', ':']:
            cleaned_text.append(char)
    return ''.join(cleaned_text)


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [10]:
dataset = dataset.map(clean_text)

Map:   0%|          | 0/9898 [00:00<?, ? examples/s]

In [11]:
dataset[2]

{'inputs': 'Một số loại hình dán có keo dính ở đằng sau số khác chỉ có một miếng vải bên dưới. Xem xét kỹ hình dán và xác định xem bạn có cần thêm vật liệu gì không. Những hình dán thêu trang trí thường dày cứng và có một lớp keo nhựa ở mặt dưới. Loại này có thể dùng để che giấu phần vải bị rách hoặc mất màu. Hình dán truyền nhiệt có hình in trên một mặt giấy và mặt kia là loại giấy không bóng. Loại này không thể vá được chỗ rách và phần vải bên dưới thường bị lộ rõ nếu không phải là vải trắng. Hình dán có mặt sau chỉ là chất liệu vải có thể gắn bằng vải keo ép. Các hình dán dùng để vá các lỗ thủng hoặc vết ố bẩn được thiết kế để hòa lẫn với vải thường có một lớp giấy đằng sau và cần được gỡ ra trước khi dán. Cân nhắc tự thiết kế hình dán nếu bạn không tìm được mẫu yêu thích. Các chất liệu như denim và cotton thường làm nền rất tốt cho các hình dán bằng bàn là. Nguyên tắc chung là chất liệu vải ít nhất phải dày bằng hình dán. Xem nhãn hướng dẫn cách chăm sóc quần áo để biết chất liệu v

In [12]:

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

Map (num_proc=8):   0%|          | 0/9898 [00:00<?, ? examples/s]

In [13]:
train_size = int(len(tokenized_dataset) * 0.8)
train_dataset = tokenized_dataset.select(range(train_size))
val_dataset = tokenized_dataset.select(range(train_size, len(dataset)))

In [14]:
dataset[1]

{'inputs': 'Hydroquinone là loại kem tẩy trắng hiệu quả có thể giúp giảm đáng kể sự xuất hiện của đồi mồi. Hydroquinone có bán ở nồng độ lên đến 2 ở dạng không kê đơn còn nồng độ cao hơn cần có đơn thuốc của bác sĩ. Cần biết rằng hydroquinone bị cấm ở nhiều quốc gia châu Âu và châu Á do đặc tính gây ung thư tiềm ẩn. Tuy nhiên ở một số quốc gia như Mỹ hydroquinone vẫn được bán rộng rãi. RetinA là sản phẩm chăm sóc da chống lão hóa tuyệt vời được dùng để chống lại nếp nhăn cải thiện độ trơn láng của da làm mờ vùng da không đều màu và tổn thương do ánh nắng bao gồm đốm đồi mồi. RetinA là dẫn xuất của vitamin A có sẵn ở dạng gel hoặc kem với nhiều nồng độ khác nhau. RetinA chỉ có sẵn ở dạng kê đơn nên bạn cần đi bác sĩ khám trước khi bắt đầu sử dụng. RetinA giúp loại bỏ đốm đồi mồi bằng cách tẩy da chết loại bỏ lớp da tăng sắc tố bên ngoài và để lộ lớp da mới tinh khôi bên dưới. Sự kết hợp của hai thành phần này đã được chứng minh là sẽ giúp làm sáng màu các vết đồi mồi. Hãy trao đổi với b

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [16]:
del tokenized_dataset

In [17]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7918
})

In [18]:
val_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1980
})

## import metric

In [19]:
import numpy as np
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize
import evaluate
rouge_score = evaluate.load("rouge")
from datasets import load_metric


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    rouge_metric = load_metric('rouge')

    rouge_metric.add_batch(predictions=decoded_preds, references=decoded_labels)
    rouge_scores = rouge_metric.compute()

    return {k: v.mid.fmeasure * 100 for k, v in rouge_scores.items()}

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Define traing args 

In [20]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import EarlyStoppingCallback

        
training_args = Seq2SeqTrainingArguments("finetune-newwiki-summarization-ver2",
                                      num_train_epochs=7,
                                      learning_rate=1e-6,
                                      warmup_ratio=0.1,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      evaluation_strategy="epoch",
                                      save_total_limit=1,
                                      fp16=True, # if GPU
                                      predict_with_generate=True,
#                                       logging_steps=200,
                                      push_to_hub=True,
                                         load_best_model_at_end=True
                                      )

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### Traning loop

In [21]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/minnehwg/finetune-newwiki-summarization-ver2 into local empty directory.
Using amp half precision backend
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7918
  Num 

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.491200,0.470079,48.175356,25.022139,34.761329,37.073443
2,0.474800,0.469357,48.362864,25.364867,35.023890,37.308376
3,0.475500,0.469549,48.276979,25.190664,34.845579,37.193011
4,0.470300,0.469566,48.180064,25.176948,34.800357,37.081658
5,0.468000,0.469694,48.165870,25.149090,34.779352,37.089311


***** Running Evaluation *****
  Num examples = 1980
  Batch size = 8
/tmp/ipykernel_34/806200219.py:17: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Saving model checkpoint to finetune-newwiki-summarization-ver2/checkpoint-990
Configuration saved in finetune-newwiki-summarization-ver2/checkpoint-990/config.json
Model weights saved in finetune-newwiki-summarization-ver2/checkpoint-990/pytorch_model.bin
tokenizer config file saved in finetune-newwiki-summarization-ver2/checkpoint-990/tokenizer_config.json
Special tokens file saved in finetune-newwiki-summarization-ver2/checkpoint-990/special_tokens_map.json
Copy vocab file to finetune-newwiki-summarization-ver2/checkpoint-990/spiece.model
tokenizer config file saved in finetune-newwiki-summarization-ver2/tokenizer_config.json
Special tokens file saved in finetune-newwiki-summarization-ver2/special_tokens_map.json
Copy vocab file to finetune-newwiki-summarization-ver2/spiece.model
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.

TrainOutput(global_step=4950, training_loss=0.47480650314176925, metrics={'train_runtime': 7487.117, 'train_samples_per_second': 7.403, 'train_steps_per_second': 0.926, 'total_flos': 4.82172857745408e+16, 'train_loss': 0.47480650314176925, 'epoch': 5.0})

## Push model to huggingface hub

In [22]:
trainer.push_to_hub()

Saving model checkpoint to finetune-newwiki-summarization-ver2
Configuration saved in finetune-newwiki-summarization-ver2/config.json
Model weights saved in finetune-newwiki-summarization-ver2/pytorch_model.bin
tokenizer config file saved in finetune-newwiki-summarization-ver2/tokenizer_config.json
Special tokens file saved in finetune-newwiki-summarization-ver2/special_tokens_map.json
Copy vocab file to finetune-newwiki-summarization-ver2/spiece.model


Upload file runs/May13_10-43-13_a81d583e2710/events.out.tfevents.1715597010.a81d583e2710.34.0:   0%|          …

To https://huggingface.co/minnehwg/finetune-newwiki-summarization-ver2
   3ae6881..52e2b4b  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 48.16587011744156}]}
To https://huggingface.co/minnehwg/finetune-newwiki-summarization-ver2
   52e2b4b..de72c49  main -> main



'https://huggingface.co/minnehwg/finetune-newwiki-summarization-ver2/commit/52e2b4b56893e446500d8927a7d3012a1f2d25a6'